In [ ]:
import napari
import os
import skimage
from skimage import io, measure
from skimage.io import imread
import numpy as np
import copy
import pandas as pd
import tifffile
from PIL import Image, ImageFont, ImageDraw 
import seaborn as sns

In [ ]:
#paths
codedir = '/Users/engje/Documents/Data/mTMA-4'
regdir = f'{codedir}/Cropped'
#regdir = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2022-03-25_mTMA2/Cropped'
os.chdir(codedir)
os.chdir('..')
from mplex_image import visualize as viz
from mplex_image import mpimage, analyze
os.chdir(codedir)
title_font = ImageFont.truetype('SansSerifBldFLF.otf',50)

In [ ]:
#load single cell data
os.chdir(codedir)
s_data = 'tma_meta.data.csv'
df_mi = pd.read_csv(f'{s_data}',index_col=0)
df_mi = df_mi.set_index('subtractedregisteredimages')
df_mi['DAPI_X'] = df_mi.umX/.325 #convert back to pixels
df_mi['DAPI_Y'] = df_mi.umY/.325
df_mi['cellid'] = [item.split('cell')[1] for item in df_mi.index]
df_prop = ((df_mi.groupby(['slidescene','seurat_clusters']).count().iloc[:,0])/(df_mi.groupby(['slidescene']).count().iloc[:,0])).unstack().fillna(0)

In [ ]:
# make the intensity data (done)
# ls_file = ['features_mTMA2-4_Ecad_patched_MeanIntensity_Shape_DAPI1_DAPI6_registeredimages.csv',
#  'features_mTMA2-5_Ecad_patched_MeanIntensity_Shape_DAPI1_DAPI6_registeredimages.csv']
# df_all = pd.DataFrame()
# for s_file in ls_file:
#     df = pd.read_csv(s_file,index_col=0)
#     df_all = pd.concat([df_all,df])
# df_all = df_all.loc[:,df_all.dtypes=='float64']
# df_all = df_all.loc[:,~df_all.columns.str.contains('DAPI')]
# df_all = df_all.loc[:,~df_all.columns.str.contains('cellmem2p25')]
# df_all.columns = [item.split('_')[0] for item in df_all.columns]
# df_all = df_all.loc[:,~df_all.columns.duplicated(keep=False)]
# df_all.drop('nuclei',axis=1,inplace=True)
# df_mi.loc[:,['seurat_clusters']].merge(df_all,left_index=True,right_index=True).to_csv('tma_int.data.csv')

#load intensity data
df_clust = pd.read_csv('tma_int.data.csv',index_col=0)

In [ ]:
%matplotlib inline 
ls_drop =  ['LamAC','RAD51', 'gH2AX','pMYC','pRPA','CD4', 'CD8','ColI']
df_plot = df_clust.drop(ls_drop,axis=1).groupby('seurat_clusters').mean()
g = sns.clustermap(df_plot,z_score=1,vmin=-2,vmax=2,cmap='viridis',figsize=(5,4),xticklabels=1,yticklabels=1,
                   dendrogram_ratio=0.1, cbar_pos=(0.01, 0.9, 0.04, 0.15))
g.savefig('clusters_heatmap.png',dpi=300)

In [ ]:
test_cluster = 1
df_prop.sort_values(by=test_cluster,ascending=False)[0:5]

## Load a core

and add seurat clustering results

In [ ]:
# crops/local
s_slide ='mTMA2-4_sceneG07' # 

#load images
print(s_slide)
os.chdir(f'{regdir}')
for s_file in os.listdir():
    if s_file.find(s_slide) > -1:
        s_crop = s_file.split('_')[2]
viewer = napari.Viewer()
label_image = viz.load_crops(viewer,s_crop,s_slide)
viewer.scale_bar.visible = True
title_text = f'{s_slide}'
viz.add_slide_name(viewer, title_text,title_font, s_layer='DAPI1')
label_corrected = viz.correct_labels(viewer)
viewer.add_labels(label_corrected)

In [ ]:
# add all the clusters
df_scene = df_mi[df_mi.slidescene==s_slide] #[:-1]
s_col = 'seurat_clusters'
for idx,i_clust in enumerate(sorted(df_scene.loc[:,s_col].unique())): 
    viz.add_cluster(viewer,df_scene,s_col,i_clust,idx,label_corrected)

## Area exclusion

select areas of the tissue to analyze or exclude

In [ ]:
# add points at centroids
points,x_arr = add_points(s_crop,df_mi,s_slide)
viewer.add_points(points)

In [ ]:
#draw shape, then run this cell (after every new shape)

verts = viewer.layers['Shapes'].data[0]
b_poly = measure.points_in_poly(points, verts)
point_properties = {
    'label': np.array(df_mi.loc[df_mi.index.str.contains(s_slide),'cellid']),
    'in_poly' : np.array(b_poly)
}
points_layer = viewer.add_points(points, properties=point_properties, face_color='in_poly',face_color_cycle=['magenta', 'black'],edge_width=0)


In [ ]:
# specify if points to exclude are ...
# in polygon (might be black or magenta)
df_exclude = x_arr.loc[b_poly]
# out of polygon (again, color can change)
#df_exclude = x_arr.loc[~b_poly]

In [ ]:
# add another ROI
# more in poly
df_exclude = pd.concat([df_exclude,x_arr.loc[b_poly]])
# more out of poly
#df_exclude = pd.concat([df_exclude,x_arr.loc[~b_poly]])

In [ ]:
# save
df_exclude.to_csv(f'exclude_{s_slide}.csv')

## Equalized display range

View multiple tissues with same markers and display range

In [ ]:
# equalized display range for making figures
ls_slide = ['mTMA2-4_sceneA04','mTMA2-4_sceneA01',]
d_marker = {'DAPI1':0,'CD31':0,'Ecad':50,'Ki67':80,'CD45':(2,30),}
os.chdir(regdir)
for s_slide in ls_slide: 
    print(s_slide)
    for s_file in os.listdir():
        if s_file.find(s_slide) > -1:
            s_crop = s_file.split('_')[2]
    viewer = napari.Viewer()
    label_image = viz.load_marker(viewer,s_crop,s_slide,d_marker)
    viewer.scale_bar.visible = True
    viz.add_slide_name(viewer, s_slide, title_font,s_layer=list(d_marker.keys())[0])
    #break

## Viewing markers at full resolution (server)

make sure to mount server (cmd-k)

requires access

In [ ]:
## viewing markers at full resolution (server)
regdir_server = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2022-03-25_mTMA2/SubtractedRegisteredImages'
segdir = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2022-03-25_mTMA2/Segmentation/mTMA2-4_CellposeSegmentation'

from skimage.io import imread
ls_rpa = ['mTMA2-4_sceneA01'
 ]

ls_marker = ['pMYC','DAPI1']
for s_slide in ls_rpa:
    regdir_slide = f'{regdir_server}/{s_slide}'
    os.chdir(regdir_slide)
    df_img = mpimage.parse_org()
    #update dapis
    d_out = {item:'DAPI' + df_img.loc[item,'round_num'].astype('int').astype('str') for item in df_img[df_img.marker=='DAPI'].index}
    for s_index, s_dapi in d_out.items():
        df_img.loc[s_index,'marker'] = s_dapi
    filenames = df_img[df_img.marker.isin(ls_marker)].index
    viewer = napari.Viewer()
    for i, filename in enumerate(filenames):
        if filename.find('Registered') > -1:
            s_index = f"Registered{filename.split('Registered')[1]}"
            s_marker = df_img.loc[s_index,'marker']
            img = imread(s_index)
            q99 = np.quantile(img,0.998)
            if s_marker.find('DAPI') > -1:
                viewer.add_image(img, contrast_limits=[1000,65000], multiscale=False, blending='additive',visible=True,name=s_marker,colormap='blue')
            elif s_marker == 'CK19' or s_marker == 'Ecad':
                viewer.add_image(img, contrast_limits=[750,q99*1.5], multiscale=False, blending='additive',visible=False,name=s_marker)
            else:
                viewer.add_image(img, contrast_limits=[0,q99*1.5], multiscale=False, blending='additive',visible=False,name=s_marker)
    os.chdir(segdir)
    labels = io.imread(f'{s_slide}_Ecad_nuc30_cell30_matched_exp5_CellSegmentationBasins.tif')
    viewer.add_labels(labels,blending='additive')
#     nuclabels = io.imread(f'{s_slide}_nuclei0.5_NucleiSegmentationBasins.tif')
#     viewer.add_labels(nuclabels,blending='additive')
    add_slide_name(viewer, s_slide, title_font,s_layer='DAPI1')
    viewer.scale_bar.visible = True
    break


###  move images around on one drive

copy cores analyzed by zinab

In [ ]:
## server (works way better)
# import shutil
# cropdir = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2022-03-25_mTMA2/Cropped'
# os.chdir(cropdir)
# ls_scene = df_mi.slidescene.unique()[2::]
# df_img = pd.DataFrame(index=os.listdir())
# df_img['scene'] = [item.split('_')[0] + '_' + item.split('_')[1] for item in df_img.index]
# for s_scene in ls_scene:
#     print(s_scene)
#     ls_img = df_img[df_img.scene==s_scene].index
#     for s_img in ls_img:
#         source = s_img
#         destination = f'{regdir}/{s_img}'
#         shutil.copy(source, destination)
#     #break
# os.chdir(codedir)

In [ ]:
## one drive (works poorly)
# ls_scene = df_mi.slidescene.unique()
# os.chdir('/Users/engje//Oregon Health & Science University - Zinab Doha - cyclic_IF_ZD_JE/Cropped1')
# import shutil
# df_img = pd.DataFrame(index=os.listdir())
# df_img['scene'] = [item.split('_')[0] + '_' + item.split('_')[1] for item in df_img.index]
# for s_scene in ls_scene:
#     print(s_scene)
#     ls_img = df_img[df_img.scene==s_scene].index
#     for s_img in ls_img:
#         source = s_img
#         destination = f'../Cropped/{s_img}'
#         shutil.copy(source, destination)
#os.chdir(codedir)